This notebook contains the developent of a custom component for home-assistant for monitoring Hue motion sensors and 4 button remotes. The component is a RESTful sensor. It is split into 2 main sections: functions to parse the json data returned by the API call, and classes to represent the sensor data in HASS.

In [1]:
import requests
import json
import pprint
from datetime import datetime, timedelta

def print_json(json_data):
    pprint.PrettyPrinter().pprint(json_data)

# for loading URL    
def load_url(filename):
    try:
        with open(filename, 'r') as fp:
            url = json.load(fp)
    except Exception as e:
        print('Failed to load url')
        url = None
    return url['url']

In [2]:
filename = '/Users/robincole/Desktop/Hue_url.json'
URL = load_url(filename)    # base URL of the remote, ending /sensors
sensors = requests.get(URL).json()

In [3]:
sensors.keys()

dict_keys(['20', '3', '7', '15', '9', '10', '14', '4', '16', '11', '21', '6', '5', '1', '12', '18', '8', '2', '13'])

Print out all my sensors

In [4]:
id_list = []
for i in range(1,25):
    try:
        type = sensors[str(i)]['type'][3:]   # First 3 characters always ZZL
        name = sensors[str(i)]['name']
        id = sensors[str(i)]['uniqueid']
        modelid = sensors[str(i)]['modelid']
        print(str(i) + ' : ' + type + ' : ' + modelid + ' : ' + name + ' : ' + id)
    except:
        pass

2 : Switch : RWL021 : Remote bedroom : 00:17:88:01:10:3e:3a:dc-02-fc00
3 : PGenericStatus : PHWA01 : Dimmer Switch 2 SceneCycle : WA0001
4 : Temperature : SML001 : Hue temperature sensor 1 : 00:17:88:01:02:00:af:28-02-0402
5 : Presence : SML001 : Hall Sensor : 00:17:88:01:02:00:af:28-02-0406
6 : LightLevel : SML001 : Hue ambient light sensor 1 : 00:17:88:01:02:00:af:28-02-0400
7 : PGenericStatus : PHA_STATE : MotionSensor 5.Companion : MotionSensor 5.Companion
8 : Temperature : SML001 : Hue temperature sensor 2 : 00:17:88:01:02:00:b5:ce-02-0402
9 : Presence : SML001 : Bedroom sensor : 00:17:88:01:02:00:b5:ce-02-0406
10 : LightLevel : SML001 : Hue ambient light sensor 2 : 00:17:88:01:02:00:b5:ce-02-0400
11 : PGenericStatus : PHA_STATE : MotionSensor 9.Companion : MotionSensor 9.Companion
12 : fence : HA_GEOFENCE : Robins iPhone : L_02_IYwHM
13 : PPresence : HOMEAWAY : HomeAway : L_01_9rQ8A
14 : Temperature : SML001 : Hue temperature sensor 3 : 00:17:88:01:02:01:aa:9c-02-0402
15 : Presen

Individual devices indicated by first 5 characters in last entry in id_unique. 
I have 3 motion sensors and 2 remotes.

Will ignore scenes, with modelid PHWA01.

Look at the data for a remote.

In [5]:
remote_index = 12
remote_url = URL + '/{}'.format(remote_index)
remote = requests.get(remote_url).json()

In [6]:
print_json(remote)

{'config': {'on': True, 'reachable': True},
 'manufacturername': 'Philips',
 'modelid': 'HA_GEOFENCE',
 'name': 'Robins iPhone',
 'recycle': False,
 'state': {'lastupdated': '2017-09-14T18:44:05', 'presence': False},
 'swversion': 'A_1',
 'type': 'Geofence',
 'uniqueid': 'L_02_IYwHM'}


In [7]:
remote['state']#['buttonevent']

{'lastupdated': '2017-09-14T18:44:05', 'presence': False}

Format the time correctly

In [8]:
lastupdated = remote['state']['lastupdated']
lastupdated

'2017-09-14T18:44:05'

In [9]:
lastupdated.split('T')

['2017-09-14', '18:44:05']

Format the name correctly

In [10]:
line='Hall Sensor'
arr = line.split()
arr.insert(1, ' Motion ')
print(' '.join(arr))

Hall  Motion  Sensor


## Parse the 3 kinds of sensor
Functions to parse 3 kinds of sensor, and a function to parse the API json data returned which calls the 3 functions.
#### Parsing the SML001 Hue motion sensor

In [11]:
def parse_SML001(response): 
    '''Parse the json response for a SML001 Hue motion sensor and return the data'''
    if 'ambient light' in response['name']:
        data = {'light level': response['state']['lightlevel']}
        
    elif 'temperature' in response['name']:
        data = {'temperature':response['state']['temperature']}
        
    else:
        # Some logic to conver 'Hall Sensor' to 'Hall Motion Sensor'
        name_raw = response['name']        
        arr = name_raw.split()
        arr.insert(-1, 'motion')
        name = ' '.join(arr)
        
        data = {'model':response['modelid'],
                'state':response['state']['presence'],
                'name':name} 
    return data

In [12]:
motion_sensor = requests.get(URL + '/{}'.format(5)).json()
parse_SML001(motion_sensor)

{'model': 'SML001', 'name': 'Hall motion Sensor', 'state': False}

In [13]:
motion_sensor = requests.get(URL + '/{}'.format(4)).json()
parse_SML001(motion_sensor)

{'temperature': 1744}

In [14]:
motion_sensor = requests.get(URL + '/{}'.format(6)).json()
parse_SML001(motion_sensor)

{'light level': 5475}

#### Parsing the RWL021 Hue remote

In [15]:
def parse_RWL021(response):
    '''Parse the json response for a RWL021 Hue remote and return the data.
       If button held for 2 seconds then a hold.'''
    # check if long or short hold
    press = str(response['state']['buttonevent'])

    if press[-1] in ['0', '2']:    # 1002, 4001 etc, check if even
        button = str(press)[0] + '_click'
    else:
        button = str(press)[0] + '_hold'
    
    data = {'model':'RWL021',
            'name':response['name'],
            'state':button,
            'last_updated':response['state']['lastupdated'].split('T')}
    return data

Removed this code for now, it checks if it has been greater than 5 seconds since last press of remote and if so sets the status to idle. However there is currently an issue that the Hue time is out by 1 hour so this doesn't work. Check why Hue time is out and add later if required.

    # Check how long since button pressed. If > seconds consider remote idle
    lastupdated = response['state']['lastupdated'].split('T')  # returns a list
    lastupdated_str = lastupdated[0] + ' ' + lastupdated[1]   # return a string
    lastupdated_obj = datetime.strptime(lastupdated_str, '%Y-%m-%d %H:%M:%S')  # convert to datetime object
    diff = datetime.now() - lastupdated_obj
    sec_since_update = diff.seconds

    # Why is API request returning wrong time zone? 
    #if diff.seconds > 5:
    #    button = 'idle'

In [16]:
remote = requests.get(URL + '/{}'.format(20)).json()
parse_RWL021(remote)

{'last_updated': ['2017-09-14', '05:14:34'],
 'model': 'RWL021',
 'name': 'Living room remote',
 'state': '4_hold'}

#### Parse the GeoFence
Hue presnce status

In [17]:
def parse_GEOFENCE(response): 
    '''Parse the json response for a GEOFENCE and return the data'''
    data = {'name': response['name'],
            'model':'Geofence',
            'state': response['state']['presence']}
    return data

In [18]:
parse_GEOFENCE(requests.get(URL + '/{}'.format(12)).json())

{'model': 'Geofence', 'name': 'Robins iPhone', 'state': False}

## Parsing the json file
Function to parse the entire json returned by the API call. Loop over all sensors and if any of the 3 sensors of interest is found, return their data

In [19]:
def parse_hue_api_response(response):
    """Take in the Hue API json response."""
    data_dict = {}    # the list of sensors, referenced by their parsed_uniqueid
    
    # Loop over all keys (1,2 etc) in API response to identify sensors and get data
    for key in response.keys():
        sensor = response[key]
        
        if sensor['modelid'] in ['RWL021', 'SML001']:
            _key = sensor['uniqueid'].split(':')[-1][0:5]
            
            if sensor['modelid'] == 'RWL021':
                data_dict[_key] = parse_RWL021(sensor)
            else:
                if _key not in data_dict.keys():
                    data_dict[_key] = parse_SML001(sensor)
                else:
                    data_dict[_key].update(parse_SML001(sensor))   # append the data
                    
        elif sensor['modelid'] == 'HA_GEOFENCE':
            data_dict['Geofence'] = parse_GEOFENCE(sensor)
            
    return data_dict

In [20]:
my_parsed_data = parse_hue_api_response(requests.get(URL).json())
my_parsed_data

{'1e-02': {'last_updated': ['2017-09-14', '05:14:34'],
  'model': 'RWL021',
  'name': 'Living room remote',
  'state': '4_hold'},
 '28-02': {'light level': 5475,
  'model': 'SML001',
  'name': 'Hall motion Sensor',
  'state': False,
  'temperature': 1744},
 '9c-02': {'light level': 16221,
  'model': 'SML001',
  'name': 'Living room motion sensor',
  'state': False,
  'temperature': 2068},
 'Geofence': {'model': 'Geofence', 'name': 'Robins iPhone', 'state': False},
 'ce-02': {'light level': 20789,
  'model': 'SML001',
  'name': 'Bedroom motion sensor',
  'state': False,
  'temperature': 1889},
 'dc-02': {'last_updated': ['2017-09-14', '20:55:06'],
  'model': 'RWL021',
  'name': 'Remote bedroom',
  'state': '4_click'}}

# HA specific development
## Data class
A class to hold the parsed data. In HA this is onject is throttled so it is only called once a second. Therfore only a single sensor object will update the data object but since the data object is shared by all sensors it will be up to date for all sensors. 

In [21]:
class HueSensorData(object):
    """Get the latest sensor data."""

    def __init__(self):
        """Initialize the object."""
        self.data = None

    # Update only once in scan interval.
    #@Throttle(SCAN_INTERVAL)
    def update(self):
        """Get the latest data"""
        response = requests.get(URL)
        if response.status_code != 200:
            _LOGGER.warning("Invalid response from API")
        else:
            self.data = parse_hue_api_response(response.json())

In [22]:
my_data = HueSensorData()
my_data.update()

In [23]:
my_data.data.keys()   # sensors are indexed by their Hue ID

dict_keys(['Geofence', '28-02', '9c-02', 'ce-02', '1e-02', 'dc-02'])

## Sensor class

Create a generic HueSensor class.

In [24]:
class HueSensor():
    """Class to hold Hue Sensor basic info."""

    ICON = 'mdi:run-fast'
        
    def __init__(self, hue_id, data):
        self._hue_id = hue_id
        self._data = data    # remember this is the data object, data is in .data
        self._name = self._data.data[self._hue_id]['name']      
        self._model = self._data.data[self._hue_id]['model']
        self._state = self._data.data[self._hue_id]['state']
        self._attributes = {}
        
    @property
    def name(self):
        """Return the name of the sensor."""
        return self._name

    @property
    def state(self):
        """Return the state of the sensor."""
        return self._state
    
    @property
    def icon(self):
        """Icon to use in the frontend, if any."""
        return self.ICON
    
    @property
    def device_state_attributes(self):
        """Only motion sensors have attributes currently, but could extend."""
        return self._attributes

    def update(self):
        """Update the sensor."""
        self._data.update()
        self._state = self._data.data[self._hue_id]['state']
        if self._model == 'SML001':
            self._attributes['light level'] = self._data.data[self._hue_id]['light level']
            self._attributes['temperature'] = self._data.data[self._hue_id]['temperature']
        elif self._model == 'RWL021':
            self._attributes['last updated'] = self._data.data[self._hue_id]['last_updated']

### Test the motion sensor

In [25]:
my_data.data['ce-02']

{'light level': 20789,
 'model': 'SML001',
 'name': 'Bedroom motion sensor',
 'state': False,
 'temperature': 1889}

In [26]:
test_hue_motion = HueSensor('ce-02', my_data)
test_hue_motion.update()
test_hue_motion._hue_id

'ce-02'

In [27]:
test_hue_motion.name

'Bedroom motion sensor'

In [28]:
test_hue_motion.state

False

In [29]:
test_hue_motion.device_state_attributes

{'light level': 20789, 'temperature': 1889}

### Test the remote

In [30]:
my_data.data['1e-02']

{'last_updated': ['2017-09-14', '05:14:34'],
 'model': 'RWL021',
 'name': 'Living room remote',
 'state': '4_hold'}

In [31]:
test_hue_remote = HueSensor('1e-02', my_data)
test_hue_remote.update()
test_hue_remote._hue_id

'1e-02'

In [32]:
test_hue_remote.name

'Living room remote'

In [33]:
test_hue_remote.state

'4_hold'

In [34]:
test_hue_remote.device_state_attributes

{'last updated': ['2017-09-14', '05:14:34']}

### Test the Geofence

In [35]:
my_data.data['Geofence']

{'model': 'Geofence', 'name': 'Robins iPhone', 'state': False}

In [36]:
test_geofence = HueSensor('Geofence', my_data)
test_geofence.update()
test_geofence._hue_id

'Geofence'

In [37]:
test_geofence.name

'Robins iPhone'

In [38]:
test_geofence.state

False

## Test setup platform
Mimic setup in HA

In [39]:
def setup_platform():
    """Set up the component."""
    data = HueSensorData()
    data.update()
    sensors = []
    for key in data.data.keys():
        sensors.append(HueSensor(key, data))
    return(sensors)
    #add_devices(sensors, True)

In [40]:
my_sensors = setup_platform()

In [41]:
for sensor in my_sensors:
    sensor.update()
    print(sensor.name)
    print(sensor.state)
    print(sensor.device_state_attributes)
    print('*****')

Robins iPhone
False
{}
*****
Hall motion Sensor
False
{'temperature': 1744, 'light level': 5475}
*****
Living room motion sensor
False
{'temperature': 2068, 'light level': 16221}
*****
Bedroom motion sensor
False
{'temperature': 1889, 'light level': 20789}
*****
Living room remote
4_hold
{'last updated': ['2017-09-14', '05:14:34']}
*****
Remote bedroom
4_click
{'last updated': ['2017-09-14', '20:55:06']}
*****
